# Observatory Distances

In this notebook, a dataframe is built based on the distances from each observatory to the Apophis in every time step.

In [1]:
import pandas as pd
import pylab as plt
import numpy as np

In [4]:
ephemeris = pd.read_csv("../ephemeris2.csv")
radio = pd.read_csv("../AllRadiotelescopes.csv")
print(radio)

                        Name                                        Location  \
0                    Arecibo                            Arecibo, Puerto Rico   
1                  Goldstone                   Mojave Desert, California, US   
2                 Green Bank                   Green Bank, West Virginia, US   
3           Very Large Array                         Socorro, New Mexico, US   
4                       FAST                                  Guizhou, China   
5            Canberra DSS-43       Tidbinbilla, Australian Capital Territory   
6                      EDSCC                                Ussuriisk, Rusia   
7   Usuda Deep Space Center                              Nagano Pref., Japan   
8                 Effelsberg  Bad M�nstereifel-Effelsberg near Bonn, Germany   
9                     Lovell     Jodrell Bank Observatory, Cheshire, England   
10             Madrid DSS-63                                          Madrid   
11                    Parkes            

## Distances

In [5]:
"ASTROPY"
from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun
from astropy.constants import c
import astropy.units as u

In [6]:
import warnings
warnings.simplefilter('ignore', UserWarning)

In [23]:
# Obtains the Astropy Position Vector of given earth coordinates
pos = lambda x,h: EarthLocation(lon=x['Longitude']*u.deg,lat=x['Latitude']*u.deg,height=h*u.earthRad)

# Transforms an Astropy object into a simple 3-tuple vector
vec = lambda a:  np.array((a.x.value,
                  a.y.value,
                  a.z.value))

# Obtains the difference between two Astropy Vectors
vecdiff = lambda a,b: (a.x.value-b.x.value,
                       a.y.value-b.y.value,
                       a.z.value-b.z.value)

# Returns the magnitude of a vector
dist = lambda v: np.linalg.norm(v)

# Calculates the angle between two vectors in degrees
angle = lambda v1,v2: np.rad2deg( np.arccos( np.dot(v1,v2) / (dist(v1) * dist(v2)) ) )

In [28]:
distances = pd.DataFrame(columns=['datetime_jd','datetime_str'])
distances[['datetime_jd','datetime_str']] = ephemeris[['datetime_jd','datetime_str']]

def distance_to_apophis(obs,row):
    pos1 = pos(obs,0)
    posA = pos(row,row['delta (Rt)'])
    vec1 = vecdiff(posA,pos1) # Vector from obs1 to Apophis
    dist1 = dist(vec1)*u.earthRad # Magnitudes
    return dist1.value

for i,obs in radio.iterrows():
    name = obs['Name']
    obs_distances = []
    for j,row in ephemeris.iterrows():
            obs_distances += [distance_to_apophis(obs,row)]
    distances[name] = obs_distances
distances

,datetime_jd,datetime_str,Arecibo,Goldstone,Green Bank,Very Large Array,FAST,Canberra DSS-43,EDSCC,Usuda Deep Space Center,Effelsberg,Lovell,Madrid DSS-63,Parkes,AVN-Ghana
0,2462240.063,2029-Apr-13 13:30,31.941016,31.383546,31.832824,31.496637,30.808688,30.191080,30.857648,30.709430,31.979115,32.000982,32.082872,30.184727,32.034655
1,2462240.066,2029-Apr-13 13:35,31.670226,31.115917,31.561815,31.229158,30.512297,29.906999,30.568492,30.421322,31.689575,31.713383,31.794780,29.900336,31.744344
2,2462240.069,2029-Apr-13 13:40,31.399186,30.848328,31.290648,30.961666,30.216104,29.623267,30.279577,30.133507,31.399803,31.425551,31.506395,29.616302,31.453672
3,2462240.073,2029-Apr-13 13:45,31.131204,30.585436,31.022781,30.698796,29.916401,29.339662,29.989479,29.844901,31.107734,31.136009,31.215977,29.332333,31.159966
4,2462240.076,2029-Apr-13 13:50,30.859560,30.317908,30.751228,30.431242,29.620703,29.056749,29.701147,29.557786,30.817438,30.847650,30.926927,29.049135,30.868476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2462240.740,2029-Apr-14 05:45,30.994371,30.728870,30.770691,30.797336,29.337427,30.158195,29.521109,29.561232,29.958390,30.036937,30.136885,30.123845,30.318439
196,2462240.743,2029-Apr-14 05:50,31.271255,31.019876,31.052198,31.086152,29.624777,30.453067,29.813360,29.855791,30.229456,30.308816,30.405548,30.419031,30.581977
197,2462240.747,2029-Apr-14 05:55,31.545674,31.312654,31.332596,31.376047,29.914056,30.752157,30.108653,30.154117,30.496889,30.577219,30.669805,30.718532,30.839819
198,2462240.750,2029-Apr-14 06:00,31.821758,31.602969,31.613381,31.664120,30.202240,31.047444,30.401392,30.449159,30.768034,30.849079,30.938422,31.014151,31.103383


In [8]:
distances.to_csv("../Data/observatory_distances_indexes.csv")

In [26]:
elevations = pd.DataFrame(columns=['datetime_jd','datetime_str'])
elevations[['datetime_jd','datetime_str']] = ephemeris[['datetime_jd','datetime_str']]

def elevation_angle(obs,row):
    pos1 = pos(obs,0)
    posA = pos(row,row['delta (Rt)'])
    vec1 = vecdiff(posA,pos1) # Vector from obs1 to Apophis
    zenith_angle = angle(vec(pos1),vec(posA)-vec(pos1))
    elevation = 90 - zenith_angle
    return elevation

for i,obs in radio.iterrows():
    name = obs['Name']
    obs_elevations = []
    for j,row in ephemeris.iterrows():
            obs_elevations += [elevation_angle(obs,row)]
    elevations[name] = obs_elevations
elevations

,datetime_jd,datetime_str,Arecibo,Goldstone,Green Bank,Very Large Array,FAST,Canberra DSS-43,EDSCC,Usuda Deep Space Center,Effelsberg,Lovell,Madrid DSS-63,Parkes,AVN-Ghana
0,2462240.063,2029-Apr-13 13:30,-51.478890,-13.522817,-42.628884,-20.241871,20.028456,76.982866,17.056446,26.316442,-55.169676,-57.385337,-67.140980,78.735742,-60.853223
1,2462240.066,2029-Apr-13 13:35,-52.563784,-14.413579,-43.531706,-21.171115,20.869098,77.355138,17.441400,26.661918,-54.507932,-56.882897,-66.379643,79.265784,-59.995323
2,2462240.069,2029-Apr-13 13:40,-53.650978,-15.309360,-44.434674,-22.104408,21.703018,77.643703,17.812920,26.989775,-53.834097,-56.362964,-65.600452,79.706912,-59.119372
3,2462240.073,2029-Apr-13 13:45,-55.062209,-16.486242,-45.614206,-23.327538,22.763656,77.905974,18.257993,27.371096,-52.956575,-55.679419,-64.577741,80.148592,-57.944448
4,2462240.076,2029-Apr-13 13:50,-56.154001,-17.392711,-46.516363,-24.269344,23.579858,77.979076,18.596894,27.656370,-52.257473,-55.122212,-63.763055,80.338941,-57.032195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2462240.740,2029-Apr-14 05:45,-43.881196,-25.641391,-28.298351,-30.013649,76.475041,7.804390,51.610234,47.921501,19.737776,14.967840,9.049244,9.806834,-1.428749
196,2462240.743,2029-Apr-14 05:50,-43.426194,-26.157127,-28.215621,-30.406984,75.341196,7.104708,50.716877,46.895704,20.464096,15.624829,9.877588,9.084382,-0.323710
197,2462240.747,2029-Apr-14 05:55,-42.784937,-26.786782,-28.061344,-30.872235,73.874075,6.162587,49.565002,45.573769,21.421014,16.494520,10.967742,8.112980,1.108925
198,2462240.750,2029-Apr-14 06:00,-42.275449,-27.262795,-27.931636,-31.219136,72.746342,5.442741,48.668233,44.553975,22.151747,17.158856,11.805398,7.370201,2.215189


In [27]:
elevations.to_csv("../Data/elevations.csv")